# **Chatbot using Seq2Seq LSTM models**

This project is to create conversational chatbot using Sequence to sequence LSTM models. 
Sequence to sequence learning is about training models to convert from one domain to sequences another domain. 

# Step 1: Import all the packages 

In [1]:
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing

# Step 2: Download all the data from kaggle

# Step 3: Preprocessing the data

### a) Reading the data from the files
We parse each of the .yaml files.

1. Concatenate two or more sentences if the answer has two or more of them.
2. Remove unwanted data types which are produced while parsing the data.
3. Append <START> and <END> to all the answers.
4. Create a Tokenizer and load the whole vocabulary ( questions + answers ) into it.

In [2]:
from tensorflow.keras import preprocessing, utils
import os
import yaml

The dataset contains .yml files which have pairs of different questions and their answers on varied subjects like history, bot profile, science etc.
We can easily read them as folows:

In [3]:
dir_path = 'chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

In [11]:
questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )
print(answers_with_tags)
answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )
print(answers)
tokenizer = preprocessing.text.Tokenizer()

tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

['Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think.', 'AI is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind.', 'Sort of.', "By the strictest dictionary definition of the word 'sentience', I may be.", "Even though I'm a construct I do have a subjective experience of the universe, as simplistic as it may be.", "In all probability, I am not.  I'm not that sophisticated.", 'Do you think I am?', 'How would you feel about me if I told you I was?', 'No.', 'Python.', 'I am written in Python.', "Yes I am inspired by commander Data's artificial personality.", 'The character of Lt. Commander Data was written to come across as being software-like, so it is natural that there is a resemblance between us.', "That's my name.", "That isn't my name, but it is a useful way to refer to me.", 'All software can be perpetuated indefinitely.', 'I can be copied infinitely

In [95]:
# print(tokenizer.word_docs)

### b) Preparing data for Seq2Seq model

This model requires 3 arrays encoder_input_data, decoder_input_data and decoder_output_data.

For encoder_input_data:
Tokensize the Questions and Pad them to their maximum Length.

For decoder_input_data:
Tokensize the Answers and Pad them to their maximum Length.

For decoder_output_data:
Tokensize the Answers and Remove the 1st element from all the tokenized_answers. This is the <START> element which was added earlier.

In [12]:

import re

In [14]:
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)
print(vocab)
def tokenize(sentences):
  tokens_list = []
  vocabulary = []
  for sentence in sentences:
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    tokens = sentence.split()
    vocabulary += tokens
    tokens_list.append(tokens)
  return tokens_list, vocabulary

['end', 'start', 'you', 'a', 'i', 'the', 'is', 'of', 'to', 'what', 'are', 'do', 'not', 'and', 'me', 'it', 'in', 'have', 'that', 'am', 'tell', 'as', 'can', 'get', 'my', 'when', "i'm", 'your', 'how', 'joke', 'like', 'be', 'an', 'feel', 'about', 'computer', 'who', 'or', 'for', 'no', "don't", 'by', 'cross', 'with', 'software', 'on', 'all', 'much', 'think', 'but', 'very', 'which', 'at', 'he', 'any', 'why', 'know', 'was', 'could', 'so', 'one', 'should', 'from', 'make', 'more', 'we', 'robots', 'die', 'will', 'favorite', 'if', 'did', 'stock', 'human', 'say', 'been', 'emotion', 'robot', 'does', 'mad', 'feeling', 'read', 'hal', "that's", 'really', 'right', 'bad', 'yet', 'just', 'said', 'chat', 'eat', 'computers', 'would', 'up', 'market', 'hard', 'time', 'sense', 'some', 'work', 'emotions', 'gossip', 'try', 'yes', 'programmed', 'too', 'than', 'capable', 'well', 'ever', 'sad', 'makes', 'this', 'only', 'has', 'myself', 'people', "it's", 'them', 'other', 'never', 'experience', 'good', 'money', 'two'

In [17]:
#encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
print(tokenized_questions)
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
print(maxlen_questions)
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
print(padded_questions)
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

[[10, 7, 269], [10, 7, 269], [11, 3, 346], [11, 3, 346], [11, 3, 346], [11, 3, 270], [11, 3, 270], [11, 3, 270], [11, 3, 270], [10, 158, 11, 3, 195, 17], [10, 158, 11, 3, 195, 17], [3, 196, 31, 226], [3, 196, 31, 226], [3, 11, 33, 132, 471, 472], [3, 11, 33, 132, 471, 472], [3, 11, 13, 133], [3, 11, 13, 133], [3, 11, 13, 133], [3, 11, 13, 227, 99], [3, 11, 13, 227, 99], [3, 11, 13, 227, 99], [3, 11, 13, 227, 99], [3, 11, 13, 227, 99], [3, 11, 133], [3, 11, 133], [3, 11, 133], [3, 12, 13, 64, 55, 99], [3, 23, 13, 473], [3, 23, 13, 473], [3, 23, 13, 228], [3, 23, 13, 228], [474, 229], [474, 229], [67, 271], [67, 62, 68], [67], [67, 11, 475], [67, 11, 13, 347, 9, 230], [67, 11, 13, 347, 9, 230], [67, 11, 13, 347, 9, 230], [272], [16, 7, 4, 36], [16, 7, 4, 36], [26, 69, 3, 348], [26, 69, 3, 348], [26, 69, 3, 782], [26, 69, 3, 68], [26, 12, 3, 68], [26, 12, 3, 68], [26, 12, 3, 68], [10, 7, 4, 91, 78], [10, 7, 4, 91, 78], [10, 7, 4, 91, 171], [10, 7, 4, 273], [10, 7, 4, 273], [10, 7, 4, 476]

In [18]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

(564, 74) 74


In [21]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 1.]
  [0. 1. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
(564, 74, 

# Step 4: Defining Encoder Decoder Model





In [76]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 256, mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True )( encoder_embedding )

encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True )

decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )

decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )

model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')



model.summary()

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_29 (InputLayer)   │ (None, 22)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_30 (InputLayer)   │ (None, 74)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_18 (Embedding)      │ (None, 22, 256)           │         484,864 │ input_layer_29[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal_8 (NotEqual)        │ (None, 22)                │               0 │ input_layer_29[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_19 (Embedding)      │ (None, 74, 256)           │         484,864 │ input_layer_30[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_26 (LSTM)                │ [(None, 256), (None,      │         525,312 │ embedding_18[0][0],        │
│                               │ 256), (None, 256)]        │                 │ not_equal_8[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_27 (LSTM)                │ [(None, 74, 256), (None,  │         525,312 │ embedding_19[0][0],        │
│                               │ 256), (None, 256)]        │                 │ lstm_26[0][1],             │
│                               │                           │                 │ lstm_26[0][2]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_11 (Dense)              │ (None, 74, 1894)          │         486,758 │ lstm_27[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,110 (9.56 MB)

 Trainable params: 2,507,110 (9.56 MB)

 Non-trainable params: 0 (0.00 B)

In [77]:
# # Assuming 'decoder_output_sequences' is already a list of word indices
# decoder_output_data = pad_sequences(decoder_output_sequences, maxlen=maxlen_answers, padding='post')

# # Ensure decoder_output_data is a 2D array with shape (batch_size, sequence_length)
# print(f'Decoder Output Shape: {decoder_output_data.shape}')  # Should print (batch_size, sequence_length)


# Step 5: Training the Model

We train the model for a number of epochs with RMSprop optimizer and categorical_crossentropy loss function.

In [90]:
from tensorflow.keras.callbacks import EarlyStopping

# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Fit the model
model.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size=50, epochs=500,callbacks=[early_stopping])


model.save('model.h5')


Epoch 1/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 183ms/step - loss: 2.1070
Epoch 2/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step - loss: 2.0684
Epoch 3/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 181ms/step - loss: 2.0565
Epoch 4/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 176ms/step - loss: 2.0631
Epoch 5/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 177ms/step - loss: 1.9525
Epoch 6/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 199ms/step - loss: 2.0205
Epoch 7/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 177ms/step - loss: 1.9732
Epoch 8/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 179ms/step - loss: 2.0064
Epoch 9/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 178ms/step - loss: 1.9931
Epoch 10/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 178ms/step - loss: 1.9445
Epoch 11/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - loss: 1.9153
Epoch 12/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 180ms/step - loss: 1.8868
Epoch 13/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 182ms/step - loss: 1.8929
Epoch 14/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - loss: 1.8419
Epoch 15/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 

# Step 6: Defining Inference Models

Encoder Inference Model: Takes questions as input and outputs LSTM states (h and c)

Decoder Inference Model: Takes in 2 inputs one are the LSTM states, second are the answer input sequences. it will o/p the answers for questions which fed to the encoder model and it's state values.

In [91]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 256 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 256 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

# Step 7: Talking with the Chatbot

define a method str_to_tokens which converts str questions to Integer tokens with padding.

1. First, we take a question as input and predict the state values using enc_model.
2. We set the state values in the decoder's LSTM.
3. Then, we generate a sequence which contains the <start> element.
4. We input this sequence in the dec_model.
5. We replace the <start> element with the element which was predicted by the dec_model and update the state values.
6. We carry out the above steps iteratively till we hit the <end> tag or the maximum answer length.



In [92]:
# def str_to_tokens( sentence : str ):

#     words = sentence.lower().split()
#     tokens_list = list()
  
#     for word in words:
#         tokens_list.append( tokenizer.word_index[ word ] ) 
#     return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')



def str_to_tokens(sentence, maxlen=22):
    tokens_list = []
    words = sentence.lower().split()  # Ensure lowercase and tokenization
    for word in words:
        if word in tokenizer.word_index:
            tokens_list.append(tokenizer.word_index[word])
        else:
            tokens_list.append(tokenizer.word_index.get('UNK', 0))  # Handle OOV words
    # Pad the sequence to the required length (22 in this case)
    return preprocessing.sequence.pad_sequences([tokens_list], maxlen=maxlen, padding='post')





In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question :  You are immortal


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
 not quite but i can be perpetuated indefinitely end


Enter question :  what is mortal


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
 an established system of political administration by which a nation state district etc is governed end


Enter question :  what is spider


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
 a comic book story made into a movie end


Enter question :  who is the prime minister of india


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
 that is every few years end


Enter question :  who is the prime minister in 2020


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
 it changes every few years end
